Extract table data from wikipedia page

In [57]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

wiki_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
res = requests.get(wiki_url)
soup = BeautifulSoup(res.content)
table = soup.find_all('table')[0]
table
df = pd.read_html(str(table))[0]
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Remove rows with Borough "Not assigned" from dataset

In [58]:
df.drop(df[df['Borough']=="Not assigned"].index, inplace=True)
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


Group dataset by Postcode and combine Neighbourhoods into comma seperated list.

In [59]:
df_combine = df.groupby("Postcode").agg({'Borough':'first','Neighbourhood':', '.join}).reset_index()
df_combine.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


If Neighbourhood is Not assigned, set Neighbourhood to Borough

In [60]:
df_combine['Neighbourhood'] = np.where(df_combine['Neighbourhood']=="Not assigned",df_combine['Borough'],df_combine['Neighbourhood'])
df_combine[df_combine['Borough']=="Queen's Park"]

,Postcode,Borough,Neighbourhood
93,M9A,Queen's Park,Queen's Park


In [61]:
df_combine.shape

(103, 3)